In [3]:
import networkx as nx
from kervis.dataset import Dataset
from kervis.kernels import ShortestPath

In [4]:
G1 = nx.Graph()
G1.add_node(0, label=0)
G1.add_node(1, label=0)
G1.add_node(2, label=0)
G1.add_node(3, label=0)
G1.add_node(4, label=1)
G1.add_node(5, label=1)
G1.add_node(6, label=1)
G1.add_edges_from([
     (0,3),
     (1,3),
     (2,3),
     (3,4),
     (4,5),
     (4,6)])

G2 = nx.Graph()
G2.add_node(7, label=0)
G2.add_node(8, label=0)
G2.add_node(9, label=0)
G2.add_node(10, label=0)
G2.add_node(11, label=1)
G2.add_node(12, label=1)
G2.add_node(13, label=1)
G2.add_edges_from([
     (7,10),
     (8,10),
     (9,10),
     (10,11),
     (11,12),
     (11,13),
     (7,12),
     (9,13)])

G = [G1, G2]

In [5]:
SP = ShortestPath()
lengths_with_labels = SP.fit_transform(G)

[(1, 6, 3), (5, 6, 2), (4, 6, 1), (1, 2, 2), (3, 4, 1), (1, 4, 2), (2, 4, 2), (0, 5, 3), (3, 5, 2), (1, 5, 3), (0, 1, 2), (2, 5, 3), (0, 6, 3), (0, 3, 1), (4, 5, 1), (1, 3, 1), (2, 6, 3), (3, 6, 2), (2, 3, 1), (0, 4, 2), (0, 2, 2)]
[(8, 9, 2), (11, 12, 1), (7, 13, 3), (8, 11, 2), (8, 12, 3), (7, 10, 1), (7, 12, 1), (9, 12, 3), (7, 9, 2), (10, 13, 2), (7, 11, 2), (9, 13, 1), (11, 13, 1), (10, 11, 1), (10, 12, 2), (7, 8, 2), (8, 13, 3), (8, 10, 1), (12, 13, 2), (9, 10, 1), (9, 11, 2)]
